In [5]:
from pyspark.sql import SparkSession, functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [3]:
spark = SparkSession.builder.appName("FindMostObscureHeroes").getOrCreate()

24/12/21 15:41:58 WARN Utils: Your hostname, Ngas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.115 instead (on interface en0)
24/12/21 15:41:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/21 15:41:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
schemas = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True)
])

In [37]:
names = spark.read.schema(schemas).option("sep", " ") .csv("../data/super_heroes/marvel_names")

In [38]:
lines = spark.read.text("../data/super_heroes/marvel_graph")

In [39]:
connections = (
    lines
    .withColumn("id", func.split(func.col("value"), " ")[0].cast(IntegerType()))
    .withColumn("connections", func.size(func.split(func.col("value"), " ")) - 1)
    .groupBy("id")
    .agg(func.sum("connections").alias("connections"))
)
connections.show()

+----+-----------+
|  id|connections|
+----+-----------+
|2142|        223|
|3749|         12|
|5803|          8|
|1580|         19|
|3918|        498|
| 496|         46|
|6336|         45|
|1342|         20|
| 833|          9|
|1645|         20|
|3997|         11|
|2866|         13|
|6357|         13|
|5518|         12|
|4519|         57|
|3794|         30|
|1959|        483|
|6397|         19|
| 463|         26|
|4101|         12|
+----+-----------+
only showing top 20 rows



In [40]:
min_connections = connections.agg(func.min("connections")).first()[0]
min_connections

1

In [41]:

# Solution 1
# Join the connections with names, order by connections and filter the ones with connections = 1
obsecure_heroes = (
    connections
    .join(names, "id")
    .orderBy(
        func.col("connections").asc()
    )
    .filter(f"connections = {min_connections}")
)

In [42]:
# Solution 2
# Filter the connections with connections = 1, join with names and order by connections
obsecure_heroes = (
    connections
    .filter(func.col("connections") == 1)
    .join(names, "id")
    .orderBy(func.col("connections").asc())
)
obsecure_heroes.show()

+----+-----------+--------------------+
|  id|connections|                name|
+----+-----------+--------------------+
|3490|          1|MARVEL BOY II/MARTIN|
|4517|          1|              RANDAK|
|5028|          1|           SHARKSKIN|
| 577|          1|              BLARE/|
|4784|          1|                RUNE|
|1089|          1|       CLUMSY FOULUP|
|2911|          1|                KULL|
|3489|          1|MARVEL BOY/MARTIN BU|
|1841|          1|              FENRIS|
|3298|          1|          LUNATIK II|
| 467|          1|        BERSERKER II|
|6411|          1|              ZANTOR|
|3014|          1|JOHNSON, LYNDON BAIN|
|2139|          1|      GIURESCU, RADU|
|4945|          1|         SEA LEOPARD|
| 835|          1|     CALLAHAN, DANNY|
|1408|          1|         DEATHCHARGE|
|4602|          1|         RED WOLF II|
|2117|          1|GERVASE, LADY ALYSSA|
+----+-----------+--------------------+



In [43]:
hero_names = obsecure_heroes.select("name")
hero_names.show()

+--------------------+
|                name|
+--------------------+
|MARVEL BOY II/MARTIN|
|              RANDAK|
|           SHARKSKIN|
|              BLARE/|
|                RUNE|
|       CLUMSY FOULUP|
|                KULL|
|MARVEL BOY/MARTIN BU|
|              FENRIS|
|          LUNATIK II|
|        BERSERKER II|
|              ZANTOR|
|JOHNSON, LYNDON BAIN|
|      GIURESCU, RADU|
|         SEA LEOPARD|
|     CALLAHAN, DANNY|
|         DEATHCHARGE|
|         RED WOLF II|
|GERVASE, LADY ALYSSA|
+--------------------+

